##Call the API with my variables for my research

In [1]:
import requests
import json
import numpy as np

# download the Demographics_Native_Country_Of_Mother
key = '&key='+'804d0a1a18d1de70764950c78e2a3a42d3d45e48'
data = 'https://api.census.gov/data/2020/cps/basic/jan?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:31' + key
print (data)
try:
  data = requests.get(data)
  print(data.status_code)
except requests.exceptions.RequestException as e:
    # Handle connection errors or HTTP errors
    print("STOP ERROR: Tools for Data Science - Semester Project: An error occurred:\n [[[[", e, "]]]]")
except AttributeError:
  print ("STOP ERROR: Please check your variable name!")

https://api.census.gov/data/2020/cps/basic/jan?tabulate=weight(PWSSWGT)&row+for&row+PEMNTVTY&row+PEMARITL&row+PESEX&row+PRCITSHP&row+GTCBSA&for=state:31&key=804d0a1a18d1de70764950c78e2a3a42d3d45e48
200


##Display a sample (a couple of lines in the top and some at the end)

In [2]:
import pandas as pd
import regex as re

##Display the first 100 and last 100 characters
data = data.text
print (data[0:100:],'\n')
print (data[-100:])

[["tabulate","state", "PEMNTVTY", "PEMARITL", "PESEX", "PRCITSHP", "GTCBSA"],
[0,"31","163","4","2", 

,"333","5","1","1","39820"],
[0,"31","168","5","1","1","39820"],
[0,"31","423","5","1","1","39820"]]


For some reason the data will return tabulate data with 0, witch means no possible match in regards to all the variables.

In [3]:
##Clean the data
pattern_cleaning = r'\[0,.*?\],?\n?' #remove the data with no tabulate information (tabulate=0)
data_clean = re.sub(pattern_cleaning, '', data)
pattern_cleaning = r'\n' #remove retunr line
data_clean = re.sub(pattern_cleaning, '', data_clean)
data_clean

'[["tabulate","state", "PEMNTVTY", "PEMARITL", "PESEX", "PRCITSHP", "GTCBSA"],[1414,"31","119","1","2","3","36540"],[1245,"31","057","1","2","3","36540"],[2381,"31","057","-1","2","3","36540"],[1657,"31","448","2","2","4","36540"],[2803,"31","207","1","2","4","36540"],[1197,"31","233","1","2","4","36540"],[1143,"31","301","1","2","4","36540"],[1413,"31","448","1","2","4","36540"],[1253,"31","210","1","2","4","36540"],[5385,"31","303","1","2","4","36540"],[1375,"31","313","6","2","4","36540"],[1651,"31","303","6","2","4","36540"],[1171,"31","139","4","1","4","36540"],[1819,"31","448","2","1","4","36540"],[1313,"31","207","1","1","4","36540"],[1840,"31","448","1","1","4","36540"],[7737,"31","303","1","1","4","36540"],[936,"31","223","1","1","4","36540"],[2021,"31","207","6","1","4","36540"],[1389,"31","215","6","1","4","36540"],[3828,"31","303","6","1","4","36540"],[1123,"31","314","2","2","5","36540"],[1725,"31","163","1","2","5","36540"],[1191,"31","238","1","2","5","36540"],[1741,"31"

In [10]:
##Separe cols from data (numbers)
pattern = r'.*?(\["tabulate".*?"GTCBSA"\]),([\s\S]*)'
matches = re.search(pattern, data_clean)
data_cols = matches.group(1)
data_vals = matches.group(2)[:-1] #removing the last char "]" at the end.
#print (data_cols)
data_vals

'[1414,"31","119","1","2","3","36540"],[1245,"31","057","1","2","3","36540"],[2381,"31","057","-1","2","3","36540"],[1657,"31","448","2","2","4","36540"],[2803,"31","207","1","2","4","36540"],[1197,"31","233","1","2","4","36540"],[1143,"31","301","1","2","4","36540"],[1413,"31","448","1","2","4","36540"],[1253,"31","210","1","2","4","36540"],[5385,"31","303","1","2","4","36540"],[1375,"31","313","6","2","4","36540"],[1651,"31","303","6","2","4","36540"],[1171,"31","139","4","1","4","36540"],[1819,"31","448","2","1","4","36540"],[1313,"31","207","1","1","4","36540"],[1840,"31","448","1","1","4","36540"],[7737,"31","303","1","1","4","36540"],[936,"31","223","1","1","4","36540"],[2021,"31","207","6","1","4","36540"],[1389,"31","215","6","1","4","36540"],[3828,"31","303","6","1","4","36540"],[1123,"31","314","2","2","5","36540"],[1725,"31","163","1","2","5","36540"],[1191,"31","238","1","2","5","36540"],[1741,"31","313","1","2","5","36540"],[1385,"31","207","1","2","5","36540"],[3832,"31",

In [12]:
data_colsList = re.findall(r'"(.*?)"', data_cols) #Transform string_column in List_column

#Transform string_row in List of Lists then Matrix
rows = re.findall(r'\[(.*?)\]', data_vals)
dataM=[]
for row in rows:
  row_list = re.findall(r'-?\d+(?:\.\d+)?', row)
  dataM.append([int(elt) for elt in row_list])

#print (rows[1]) #TEST
#print (dataM[1]) #TEST

##Definition of variables:
* tabulate: represents the information related to population counts.
* state: column provides information about the state
1. PEMNTVTY Demographics-native country of mother
2. PEMARITL Demographics-marital status
3. PESEX Demographics-sex
4. PRCITSHP Demographics-United States citizenship group
5. GTCBSA Demographics-city level

We will understand the population composition by state about how mixed it is based on the native country of mothers, marital status distribution, gender demographics, citizenship status, at the city-level characteristics. This data can be used to uncover trends, patterns, and insights into the social and cultural fabric of the US population.

In [13]:
##Convert the result to a Df
df = pd.DataFrame(dataM, columns=data_colsList)
df.head()

,tabulate,state,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,GTCBSA
0,1414,31,119,1,2,3,36540
1,1245,31,57,1,2,3,36540
2,2381,31,57,-1,2,3,36540
3,1657,31,448,2,2,4,36540
4,2803,31,207,1,2,4,36540


In [16]:
##Building the YYYYMM column
df['YYYYMM'] = pd.Series(['202001' for _ in range(df.shape[0])])

##This 'df' will be called **mix_composition**

In [18]:
df.tail()

,tabulate,state,PEMNTVTY,PEMARITL,PESEX,PRCITSHP,GTCBSA,YYYYMM
72,62260,31,57,-1,1,1,36540,202001
73,1269,31,119,6,1,1,36540,202001
74,1646,31,210,6,1,1,36540,202001
75,1457,31,303,6,1,1,36540,202001
76,84779,31,57,6,1,1,36540,202001


#class creation
Now we have all we need for our analysis, let's try to reorganize what we build so far into a class called _MixComposition_

In [ ]:
class MixComposition:
  """
This class will pull the Mix composition about the US population from one state.
Code	    Label
tabulate  Counts of instances
PEMNTVTY	Demographics-native country of mother
PEMARITL	Demographics-marital status
PESEX	    Demographics-sex
PRCITSHP	Demographics-United States citizenship group
GTCBSA	  Demographics-city level
STATE	    FIPS STATE Code
GTCBSA	  Demographics-city level

  """
